# SGED-OFPOSDU — Quick Start Notebook

This notebook demonstrates how to load, explore, and visualize the **SGED-OFPOSDU** dataset (synthetic GHG emissions mapped to **OFP** and **OSDU**).

**What you'll see here:**
1. Load the dataset (from `data/` if present, otherwise from the CSV filename).
2. Basic schema checks (required columns, non-null ratios, simple ranges).
3. A couple of quick visualizations (e.g., Scope 1 vs Scope 2 by facility, emissions over time).

> Tip: Install requirements if needed:
> ```bash
> pip install pandas matplotlib
> ```

*Notebook created on 2025-08-16.*


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Charting rules for this project:
# - Use matplotlib (no seaborn)
# - Single chart per figure (no subplots)
# - Do not set explicit colors or styles unless needed for clarity


In [ ]:
# Try common repo paths; fall back to local filename if running ad hoc
possible_paths = [
    "data/SGED_OFPOSDU_v1.0.csv",
    "dataset/SGED_OFPOSDU_v1.0.csv",
    "SGED_OFPOSDU_v1.0.csv",
    "GHG_OFP_OSDU_Final32_500_V1.0.csv"  # fallback filename used earlier
]

csv_path = None
for p in possible_paths:
    if os.path.exists(p):
        csv_path = p
        break

if csv_path is None:
    raise FileNotFoundError("Could not find the dataset. Place the CSV in ./data or ./dataset and rerun.")

print(f"Using dataset: {csv_path}")
df = pd.read_csv(csv_path)
df.head()


## Basic Schema Checks

In [ ]:
required_cols = [
    "Facility_ID","Facility_Name","Facility_Type",
    "Latitude","Longitude","Reporting_Year","Month",
    "Emission_Type","Emission_Value_tCO2e",
    "Scope_1_tCO2e","Scope_2_tCO2e","Scope_3_tCO2e",
    "Activity_Type","Production_Volume","Production_Unit"
]

missing = [c for c in required_cols if c not in df.columns]
print("Missing required columns:", missing)

print("\nRow count:", len(df))
print("Column count:", len(df.columns))

# Non-null ratios
non_null = df.notna().mean().sort_values(ascending=True)
non_null.head(10), non_null.tail(10)


## Plausibility Checks (simple heuristics)

In [ ]:
checks = {}

# Emissions should be non-negative
for col in ["Emission_Value_tCO2e","Scope_1_tCO2e","Scope_2_tCO2e","Scope_3_tCO2e"]:
    if col in df.columns:
        neg_count = (df[col] < 0).sum()
        checks[col] = {"negatives": int(neg_count)}

# Lat/Long ranges
if "Latitude" in df.columns:
    checks["Latitude_range"] = {"min": float(df["Latitude"].min()), "max": float(df["Latitude"].max())}
if "Longitude" in df.columns:
    checks["Longitude_range"] = {"min": float(df["Longitude"].min()), "max": float(df["Longitude"].max())}

checks


## Plot: Scope 1 vs Scope 2 by Facility (Top 20 by total)

In [ ]:
if {"Facility_Name","Scope_1_tCO2e","Scope_2_tCO2e"}.issubset(df.columns):
    top = (df.groupby("Facility_Name")[["Scope_1_tCO2e","Scope_2_tCO2e"]]
             .sum()
             .sort_values(by=["Scope_1_tCO2e","Scope_2_tCO2e"], ascending=False)
             .head(20))
    ax = top.plot(kind="bar", figsize=(10,5))
    ax.set_title("Scope 1 vs Scope 2 by Facility (Top 20)")
    ax.set_xlabel("Facility")
    ax.set_ylabel("tCO2e")
    plt.tight_layout()
    plt.show()
else:
    print("Required columns not present for this plot.")


## Plot: Total Emissions Over Time (Year-Month)

In [ ]:
if {"Reporting_Year","Month","Emission_Value_tCO2e"}.issubset(df.columns):
    ym = df["Reporting_Year"].astype(str) + "-" + df["Month"].astype(int).astype(str).str.zfill(2)
    s = df.assign(YearMonth=ym).groupby("YearMonth")["Emission_Value_tCO2e"].sum()
    s = s.sort_index()
    plt.figure(figsize=(10,4))
    s.plot()
    plt.title("Total Emissions Over Time (sum of Emission_Value_tCO2e)")
    plt.xlabel("Year-Month")
    plt.ylabel("tCO2e")
    plt.tight_layout()
    plt.show()
else:
    print("Required columns not present for this plot.")


### Join Keys
- Use `Facility_ID` (or `Facility_Name` as a fallback) to join with OSDU Facility master data.
- `Asset` can be used to join with OSDU Asset master data if present.
- For emissions analytics, pair `Emission_Type` with `Emission_Value_tCO2e` and the scope fields.
